In [ ]:
from numpy import array
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import Callback
import time
import pickle

In [ ]:

import sys
sys.path.append('/home/jovyan/work/code/tutorial-documentation-master/code/research/data_And_DataHandlingFiles/')
from load_data import load_data

# setting parameters
# Select either the 2D state (theta/omega), or the 2D images

system = '2D-State'
#system = 'image'
#resolution = '32'
resolution = '256'
xmax = 3
xmax_labels = 2
#Training
BATCH_SIZE = 128
EPOCHS = 10
NUM_HISTORICAL_STEPS_input = 20
NUM_HISTORICAL_STEPS_output = 1

#data_folder = 'Pendulum/3D_stateData_amplitude_max-0_linearPendulum-500-steps'
#data_folder = 'Pendulum/3D_stateData_amplitude_max-0_nonLinearPendulum'
#data_folder = 'Pendulum/3D_stateData_amplitude_max-0.7_nonLinearPendulum-thirdOfSamples'

# Soft-Pendulum
#data_folder = 'Soft-pendulum/50_historicSteps_smoothedData'
data_folder = 'soft-pendulum/50_timesteps_V2'

data_directory = '/home/jovyan/work/code/tutorial-documentation-master/code/research/data_And_DataHandlingFiles/' + data_folder
#data_directory = '/home/jovyan/data/data/' + data_folder

[train_generator, valid_generator, test_generator] = load_data(system,
                                                                input_start_index = 0,
                                                                output_start_index = NUM_HISTORICAL_STEPS_input-1,
                                                                NUM_HISTORICAL_STEPS_input=NUM_HISTORICAL_STEPS_input,
                                                                NUM_HISTORICAL_STEPS_output=NUM_HISTORICAL_STEPS_output, 
                                                                directory=data_directory,
                                                                BATCH_SIZE=BATCH_SIZE,
                                                                resolution=resolution,
                                                                xmax=xmax,
                                                                xmax_labels=xmax_labels,train_states=False)


input_shape = train_generator[0][0][0].shape

In [ ]:
print(train_generator[0][0].shape)
print(train_generator[0][1].shape)

In [ ]:
# LSTM name
import os
name = '{}/LSTM_Model_epochs_{}_HistoricalSteps_{}'.format(data_folder,EPOCHS,NUM_HISTORICAL_STEPS_input)
save_path = '/home/jovyan/work/code/tutorial-documentation-master/code/research/Models/Simple_LSTM/' + name
os.makedirs(save_path,exist_ok=True)

In [ ]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
        
time_callback = TimeHistory()

In [ ]:
EVALUATION_INTERVAL = len(train_generator._indices) // BATCH_SIZE
VALIDATION_STEPS = len(valid_generator._indices) // BATCH_SIZE

In [ ]:
# Model returning only one step prediction

from tensorflow.keras.models import Model
from keras.layers import Input,Reshape

x_in = Input(input_shape)
x = LSTM(NUM_HISTORICAL_STEPS_input, activation='relu', input_shape=input_shape)(x_in)# Removed return sequences so it just returns the next step
x = Dense(NUM_HISTORICAL_STEPS_input, activation='relu')(x)
x = Dense(xmax_labels)(x)
x = Reshape(target_shape=(NUM_HISTORICAL_STEPS_output,xmax_labels))(x) # This just adds axis=1 to make it the right shape for the output
model = Model(x_in, x)
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
history = model.fit(train_generator, epochs=EPOCHS, 
                validation_data=valid_generator,
                validation_steps=VALIDATION_STEPS,
                steps_per_epoch=EVALUATION_INTERVAL,
                verbose=1,callbacks=[time_callback])
model.save(save_path)
pickle.dump( time_callback.times, open(save_path+'/times.pickle', "wb" ) )
pickle.dump( history.history, open(save_path+'/history.pickle', "wb" ) )
model.summary()

In [ ]:
#name = 'LSTM_Model_3-Layer_epochs_10'
history = pickle.load( open(save_path+'/history.pickle'.format(system), "rb" ) )
# training loss
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.figure()
plt.plot((history['loss']), 'r')

plt.legend(['loss'])
plt.yscale('log')

In [ ]:
## Check details of LSTM

In [ ]:
model2 = load_model(save_path)

In [ ]:
yhat = model2.predict(train_generator[0][0])
fig = plt.figure()
for i in range(len(train_generator[0][0])):
    plt.plot(range(NUM_HISTORICAL_STEPS_input),train_generator[0][0][i,:,0],'-b',label='angle @ time t')
    plt.plot(range(NUM_HISTORICAL_STEPS_input-len(train_generator[0][1][i])+1,NUM_HISTORICAL_STEPS_input+1),train_generator[0][1][i,:,0],'--r',label='angle @ time t+1')
    plt.plot(range(NUM_HISTORICAL_STEPS_input-len(yhat[i])+1,NUM_HISTORICAL_STEPS_input+1),yhat[i,:,0],'.g',label='predicted angle @ time t+1')

In [ ]:
index =0
X = yhat[index]
fig = plt.figure(figsize=(15,10))
plt.plot(range(len(train_generator[0][0][index])),train_generator[0][0][index,:,0],label=r'$\theta$')
plt.plot(range(len(train_generator[0][0][index])),train_generator[0][0][index,:,1],label=r'$\omega$')
plt.plot(range(len(train_generator[0][0][index])),train_generator[0][0][index,:,2],label=r'control (u)')

plt.plot(range(NUM_HISTORICAL_STEPS_input,NUM_HISTORICAL_STEPS_input+len(X)),X[:,0],'.g',label=r'$\theta$')
plt.plot(range(NUM_HISTORICAL_STEPS_input,NUM_HISTORICAL_STEPS_input+len(X)),X[:,1],'.g',label=r'$\omega$')

plt.xlabel('time')
plt.title('Comparison: Original vs ' + name)